# Import dependencies

In [2]:
import jupyter_manim
import itertools as it

from manim import *

In [3]:
class Grid(VGroup):
  def __init__(self, width, height, stroke_width=0.5, size=0.25, color=WHITE):
    super().__init__()
    self.cols = width
    self.rows = height
    
    square = Square(side_length=size)
    square.set_stroke(color=color, width=stroke_width)
    
    for h in range(height):
      for w in range(width):
        group = VGroup()
        group.add(square.copy().shift(size * (LEFT * w + DOWN * h)))
        
        self.add(group)
        
    self.move_to(ORIGIN)
  
  def fill(self, array):
    for h in range(len(array)):
      for w in range(len(array[h])):
        self[h * len(array[h]) + w].set_fill(color=WHITE, opacity=array[h][w])
  
  def get_cols(self):
    return self.cols
  
  def get_rows(self):
    return self.rows
  
  

In [42]:
kernel_values = [
  [ 0, 0.5, 0 ],
  [ 0.5,1, 0.5 ],
  [ 0, 0.5, 0 ],
]

output_data = [
  [ 1, 0.5, 0.5, 0.5, 0.5, 1 ],
  [ 0.5, 1, 0, 0, 1, 0.5 ],
  [ 0.5, 0, 1, 1, 0, 0.5 ],
  [ 0.5, 0, 1, 1, 0, 0.5 ],
  [ 0.5, 1, 0, 0, 1, 0.5 ],
  [ 1, 0.5, 0.5, 0.5, 0.5, 1 ],
]

In [108]:
%%manim --verbosity=ERROR -qh GestX_Animation

class GestX_Animation(Scene):
  def construct(self):
    
    # Introduction
    title = MathTex(r"\text{GestX}", font_size=100)
    
    self.play(Write(title))
    self.play(title.animate.shift(LEFT))
    
    side_title = MathTex(r"\text{Model}", font_size=50, color=GRAY)
    side_title.next_to(title, RIGHT)
    
    self.play(Write(side_title))
    
    group = VGroup()
    group.add(title)
    group.add(side_title)
    
    subtitle = MathTex(r"\text{Animation created by Maverick Fabroa}", font_size=32, color=GRAY)
    subtitle.next_to(group, DOWN)
    
    self.play(Write(subtitle))
    self.wait(1)
    self.play(Unwrite(group), Unwrite(subtitle))
    self.wait(2)
    
    # Part 1
    grid = Grid(8, 8, size=0.5)
  
    brace1 = BraceLabel(grid, brace_direction=UP, text="8")
    brace2 = BraceLabel(grid, brace_direction=LEFT, text="8")
    
    self.play(Create(grid))
    self.wait(2)
    self.play(Circumscribe(grid), run_time=2)
    self.wait(2)
    self.play(Create(brace1), Create(brace2))
    self.wait(2)
    
    group = VGroup()
    group.add(grid)
    group.add(brace1)
    group.add(brace2)
    
    self.play(group.animate.shift(LEFT * 3))
    self.play(group.animate.shift(DOWN))
    
    title = MathTex(r"\text{Convolution Operation}", font_size=50)
    title.move_to(UP * 3)
    
    self.play(Write(title))
    
    kernel = Grid(3, 3, size=0.5, stroke_width=1, color=YELLOW)
    kernel.shift(RIGHT * 3)
    kernel.shift(DOWN / 2)
    
    self.play(Create(kernel))
    self.play(Circumscribe(kernel))
    self.wait(1)
    
    kernel_text = MathTex(r"3\times3\text{ Kernel}", color=YELLOW, font_size=32)
    kernel_text.next_to(kernel, UP)
    
    self.play(Write(kernel_text))
    self.wait(1)
    
    for h in range(kernel.get_rows()):
      for w in range(kernel.get_cols()):
        text = Text(str(kernel_values[h][w]), font_size=18, color=YELLOW)
        text.move_to(kernel[h * kernel.get_cols() + w].get_center())
        kernel[h * kernel.get_cols() + w].add(text)
        
        self.play(Write(text), run_time=0.1)
        
    self.wait(3)
    
    self.play(Unwrite(kernel_text))
    self.play(Uncreate(brace1), Uncreate(brace2))
    self.play(grid.animate.shift(UP / 2))
    
    self.play(kernel.animate.move_to(grid[6 + grid.get_rows()].get_center()))
    
    output_grid = Grid(6, 6, color=GRAY, size=0.5)
    output_grid.shift(RIGHT * 3)
    output_grid.shift(DOWN / 2)
    
    output_grid_title = MathTex(r"\text{Output}", font_size=32, color=GRAY)
    output_grid_title.next_to(output_grid, UP)
    
    self.play(Create(output_grid), Write(output_grid_title))
    
    # Slow and highlight conv
    for w in range(output_grid.get_cols()):
      for y in range(kernel.get_rows()):
        for x in range(kernel.get_cols()):
          cell = kernel[x * kernel.get_cols() + (kernel.get_cols() - 1 - y)]
          self.play(cell.animate.set_fill(color=GREEN, opacity=0.5), run_time=0.1)
          
      output = MathTex(output_data[0][w], color=GREEN, font_size=20)
      output.move_to(output_grid[output_grid.get_cols() - w - 1].get_center())
          
      self.play(LaggedStart(FadeTransform(kernel.copy(), output_grid[output_grid.get_cols() - w - 1]), Write(output), lag_ratio=0.5))
      
      for y in range(kernel.get_rows()):
        for x in range(kernel.get_cols()):
          cell = kernel[x * kernel.get_cols() + (kernel.get_cols() - 1 - y)]
          self.play(cell.animate.set_fill(color=GREEN, opacity=0), run_time=0.01)
      
      if w < output_grid.get_cols() - 1:
        self.play(kernel.animate.shift([ 0.5, 0, 0 ]), run_time=0.5)
        
    self.wait(1)
    
    for h in range(output_grid.get_rows()):
      if h == 0: continue
      
      for w in range(output_grid.get_cols()):
        output = MathTex(output_data[h][w], color=GREEN, font_size=20)
        output.move_to(output_grid[(output_grid.get_cols() * (h + 1)) - w - 1])
        
        self.play(Write(output), run_time=0.01)
        
      output = MathTex(output_data[output_grid.get_cols() - 1][output_grid.get_cols() - 1], color=GREEN, font_size=20)
      output.move_to(output_grid[(output_grid.get_cols() * h) + 1])
      self.play(Write(output), run_time=0.01)
      
    self.play(Uncreate(grid), Uncreate(kernel))
    self.wait(3)
    
    formula = MathTex(r"C_{out} = \frac{W - K + P}{S} + 1")
    formula.shift(LEFT * 3)
    
    self.play(Write(formula), run_time=2)
    self.wait(3)
    self.play(Unwrite(formula))
    self.wait(1)


Manim Community v0.18.1

In [ ]:
CONFIG = {
    "neuron_radius" : 0.15,
    "neuron_to_neuron_buff" : MED_SMALL_BUFF,
    "layer_to_layer_buff" : LARGE_BUFF,
    "neuron_stroke_color" : BLUE,
    "neuron_stroke_width" : 3,
    "neuron_fill_color" : GREEN,
    "edge_color" : GREY_B,
    "edge_stroke_width" : 2,
    "edge_propogation_color" : YELLOW,
    "edge_propogation_time" : 1,
    "max_shown_neurons" : 8,
    "brace_for_large_layers" : True,
    "average_shown_activation_of_large_layer" : True,
    "include_output_labels" : False,
}

class NetworkMobject(VGroup):
    def __init__(self, data, **kwargs):
        VGroup.__init__(self, **kwargs)
        self.layer_sizes = data["sizes"]
        self.add_neurons()
        self.add_edges()

    def add_neurons(self):
        layers = VGroup(*[
            self.get_layer(size)
            for size in self.layer_sizes
        ])
        layers.arrange(RIGHT, buff = CONFIG["layer_to_layer_buff"])
        self.layers = layers
        self.add(self.layers)
        if CONFIG["include_output_labels"]:
            self.add_output_labels()

    def get_layer(self, size):
        layer = VGroup()
        n_neurons = size
        if n_neurons > CONFIG["max_shown_neurons"]:
            n_neurons = CONFIG["max_shown_neurons"]
        neurons = VGroup(*[
            Circle(
                radius = CONFIG["neuron_radius"],
                stroke_color = CONFIG["neuron_stroke_color"],
                stroke_width = CONFIG["neuron_stroke_width"],
                fill_color = CONFIG["neuron_fill_color"],
                fill_opacity = 0,
            )
            for x in range(n_neurons)
        ])   
        neurons.arrange(
            DOWN, buff = CONFIG["neuron_to_neuron_buff"]
        )
        for neuron in neurons:
            neuron.edges_in = VGroup()
            neuron.edges_out = VGroup()
        layer.neurons = neurons
        layer.add(neurons)

        if size > n_neurons:
            dots = MathTex("\\vdots")
            dots.move_to(neurons)
            VGroup(*neurons[:len(neurons) // 2]).next_to(
                dots, UP, MED_SMALL_BUFF
            )
            VGroup(*neurons[len(neurons) // 2:]).next_to(
                dots, DOWN, MED_SMALL_BUFF
            )
            layer.dots = dots
            layer.add(dots)
            if CONFIG["brace_for_large_layers"]:
                brace = Brace(layer, LEFT)
                brace_label = brace.get_tex(str(size))
                layer.brace = brace
                layer.brace_label = brace_label
                layer.add(brace, brace_label)

        return layer

    def add_edges(self):
        self.edge_groups = VGroup()
        for l1, l2 in zip(self.layers[:-1], self.layers[1:]):
            edge_group = VGroup()
            for n1, n2 in it.product(l1.neurons, l2.neurons):
                edge = self.get_edge(n1, n2)
                edge_group.add(edge)
                n1.edges_out.add(edge)
                n2.edges_in.add(edge)
            self.edge_groups.add(edge_group)
        self.add_to_back(self.edge_groups)

    def get_edge(self, neuron1, neuron2):
        return Line(
            neuron1.get_center(),
            neuron2.get_center(),
            buff = CONFIG["neuron_radius"],
            stroke_color = CONFIG["edge_color"],
            stroke_width = CONFIG["edge_stroke_width"],
        )

    def get_active_layer(self, layer_index):
        layer = self.layers[layer_index]
        self.activate_layer(layer)
        return layer

    def activate_layer(self, layer):
        for neuron in layer.neurons:
            neuron.set_fill(
                color = CONFIG["neuron_fill_color"],
            )
            
        return layer

    def add_output_labels(self):
        self.output_labels = VGroup()
        for n, neuron in enumerate(self.layers[-1].neurons):
            label = MathTex(str(n))
            label.set_height(0.75*neuron.get_height())
            label.move_to(neuron)
            label.shift(neuron.get_width()*RIGHT)
            self.output_labels.add(label)
        self.add(self.output_labels)